In [1]:
!git clone https://github.com/HusseinMansourMohd/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification

Cloning into 'Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification'...
remote: Enumerating objects: 633, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 633 (delta 14), reused 18 (delta 8), pack-reused 609
Receiving objects: 100% (633/633), 153.32 KiB | 10.22 MiB/s, done.
Resolving deltas: 100% (377/377), done.


In [2]:
!nvidia-smi

Tue Aug  1 14:47:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import torchvision

torch.cuda.is_available()
print(torch.__version__)
print(torchvision.__version__)

2.0.1+cu118
0.15.2+cu118


In [4]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
import torchvision.datasets as dsets
from torchsummary import summary

In [5]:
%cd /content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification

/content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification


In [6]:
!pip install transformers
!pip install torchsummary
!pip install matplotlib
!pip install timm
!pip install einops==0.6.1
!pip install scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 819.0 kB/s eta 0:00:00


In [7]:
import MedVit_Adapter
print(dir(MedVit_Adapter))

['ConvBNReLU', 'ECB', 'F', 'InteractionBlock', 'LTB', 'MedViT_adapter_base', 'MedViT_adapter_large', 'MedViT_adapter_small', 'MedVit_adapter', 'NORM_EPS', 'PatchEmbed', 'SpatialPriorModule', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'checkpoint', 'deform_inputs', 'nn', 'np', 'partial', 'register_model', 'torch']


In [8]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
import torchvision.datasets as dsets
from torchsummary import summary

In [9]:
from MedVit_Adapter import MedViT_adapter_small as small

In [10]:
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=4d317a01e1cb60bb38d1fd35b24093651851a9316f6dfd17d40627f665b5b747
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [11]:
import medmnist
from medmnist import INFO
data_flag = 'retinamnist'
# [tissuemnist , pathmnist, chestmnist, dermamnist, octmnisr ,pnemonismnist , retinamnist, bloodmnist, tissuemnist, organcmist, organs ]
download = True

NUM_EPOCHS = 50
BATCH_SIZE = 15
LR = 0.01

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [12]:
import torchvision.transforms as transforms
#preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image:image.convert('RGB')),
    #transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
]
)
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

In [13]:
import torch.utils.data as data

# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
val_dataset = DataClass(split='val', transform=train_transform,download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)


# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

100%|██████████| 3291041/3291041 [00:09<00:00, 359536.86it/s]


Using downloaded and verified file: /root/.medmnist/retinamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz


In [14]:
print(train_dataset)
print("+++++++++++++++++")
print(test_dataset)

Dataset RetinaMNIST (retinamnist)
    Number of datapoints: 1080
    Root location: /root/.medmnist
    Split: train
    Task: ordinal-regression
    Number of channels: 3
    Meaning of labels: {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4'}
    Number of samples: {'train': 1080, 'val': 120, 'test': 400}
    Description: The RetinaMNIST is based on the DeepDRiD challenge, which provides a dataset of 1,600 retina fundus images. The task is ordinal regression for 5-level grading of diabetic retinopathy severity. We split the source training set with a ratio of 9:1 into training and validation set, and use the source validation set as the test set. The source images of 3×1,736×1,824 are center-cropped and resized into 3×28×28.
    License: CC BY 4.0
+++++++++++++++++
Dataset RetinaMNIST (retinamnist)
    Number of datapoints: 400
    Root location: /root/.medmnist
    Split: test
    Task: ordinal-regression
    Number of channels: 3
    Meaning of labels: {'0': '0', '1': '1', '2': '2

In [15]:
n_class = 5
model = small(num_classes=n_class)

In [16]:
#define loss function and optimizer
if task == 'multi-task, binaty-class':
    criterion = nn.BCEWihtLogLoss()
else:
    criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [17]:
criterion


CrossEntropyLoss()

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
from torch.nn.functional import softmax
from tqdm import tqdm
# training
for epoch in range(NUM_EPOCHS):

  model = model.to(device)
  model.train()

  for inputs, targets in tqdm(train_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(torch.float32))

    if task == 'multi-label, binary-class':
      targets = targets.to(torch.float32).unsqueeze(1)
    else:
      targets = targets.to(torch.long)
      targets = targets.view(-1)


    predicted_classes = torch.argmax(outputs, dim=1)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    # Delete tensors to free up memory
    del inputs, targets, outputs, predicted_classes
    # Empty the cache to clear up some more memory
    torch.cuda.empty_cache()

  0%|          | 0/72 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 72/72 [00:05<00:00, 12.21it/s]


In [21]:
from sklearn.metrics import roc_auc_score, accuracy_score

# switch to evaluation mode
def test(split):
    model.eval()
    # Lists to store actual and predicted values
    actuals = []
    probas = []
    predictions = []

    if split == 'val':
          data_loader = train_loader_at_eval
    else:
          data_loader = test_loader
    with torch.no_grad():
        for inputs, targets in data_loader:

              inputs, targets = inputs.to(device), targets.to(device)

              if task == 'multi-label, binary-class':
                  targets = targets.to(torch.float32).unsqueeze(1)
              else:
                  targets = targets.to(torch.long)
                  targets = targets.view(-1)
              if inputs.to(torch.float32).shape[0] != 15:
                break
              outputs = model(inputs.to(torch.float32))

              softmax_outputs = softmax(outputs, dim=1)
              probas.extend(softmax_outputs.detach().cpu().numpy())

              _, predicted_classes = torch.max(outputs, 1)
              predictions.extend(predicted_classes.cpu().numpy())
              actuals.extend(targets.cpu().numpy())

    print('\n')
    print(actuals[-15:])
    print(predicted_classes)
    auc = roc_auc_score(actuals, probas, multi_class='ovr')
    accuracy = accuracy_score(actuals, predictions)
    print('AUC of the model:', auc)
    print('Accuracy of the model:', accuracy)

print('==> Evaluating...')
test('val')
test('test')


==> Evaluating...


[2, 0, 2, 0, 2, 2, 0, 3, 1, 0, 3, 3, 0, 0, 0]
tensor([2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
AUC of the model: 0.4900896005201115
Accuracy of the model: 0.10833333333333334


[3, 3, 0, 2, 4, 1, 0, 0, 2, 0, 0, 2, 0, 3, 0]
tensor([0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 4, 0, 0, 0, 4], device='cuda:0')
AUC of the model: 0.5017970641414882
Accuracy of the model: 0.23333333333333334
